<h1>Transform Data</h1>

In [25]:
import warnings
warnings.filterwarnings('ignore')

# Extract
import pandas as pd
df_participant = pd.read_csv('../Dataset/dqthon-participants.csv')

In [26]:
# Transform Bagian I - Kode Pos
# membuat kolom baru lalu memuat informasi yang diambil dari kolom 'addres'

# berada di paling akhir dari alamat
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$')

In [27]:
# Transform Bagian II - Kota
# membuat kolom baru lalu memuat informasi yang diambil dari kolom 'addres'

# kota merupakan sekumpulan karakter yang terdapat setelah nomor jalan diikuti dengan \n (newline character) atau dalam bahasa lainnya yaitu enter.
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')

In [28]:
# Transform Bagian III - Github
# membuat kolom baru lalu memuat informasi yang diambil dari kolom first_name dan last_name
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

In [32]:
# Transform Bagian IV - Nomor Handphone
# melakukan cleansing pada data nomor handphone agar memiliki format yang sama.

# Jika awalan nomor HP berupa angka 62 atau +62 yang merupakan kode telepon Indonesia, maka diterjemahkan ke 0.
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')

# Tidak ada tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')

# Tidak ada spasi pada nomor HP
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')

In [38]:
# Transform Bagian V - Nama Tim
# nama tim merupakan gabungan nilai dari kolom first_name, last_name, country dan institute.

def func(col):
    # Singkatan dari Nama Depan dan Nama Belakang dengan mengambil huruf pertama
    abbrev_name = "%s%s"%(col['first_name'][0], col['last_name'][0])
    
    # mengambik dari kolom 'country'
    country = col['country']
    
    # Singkatan dari value di kolom institute
    abbrev_institue = "%s"%(''.join(list(map(lambda word: word[0], col['institute'].split()))))
    
    return "%s-%s-%s"%(abbrev_name, country, abbrev_institue)

df_participant['team_name'] = df_participant.apply(func, axis=1)

In [50]:
# Transform Bagian VI - Email

def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split())))
    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1:
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower, last_name_lower, institute, country)
    
    return "%s%s@%s.com"%(first_name_lower, last_name_lower, institute)

df_participant['email'] = df_participant.apply(func, axis=1)

In [60]:
# Transform Bagian VII - Tanggal Lahir
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')

In [62]:
# Transform Bagian VII - Tanggal Daftar Kompetisi
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')